# Prebuilt Langgraph Agent

> This guide shows you how to set up and use LangGraph's prebuilt, reusable components, which are designed to help you construct agentic systems quickly and reliably.

__Note:__ This notebook is used to demonstrate an issue with databricks experiments when:
* The repo is a public git repo
* The experiment is stored in the git folder
* autolog feature is used (recommended way)

In [0]:
!pip install -U --quiet langgraph
!pip install -U databricks-langchain
%restart_python

## Configure an LLM¶
To configure an LLM with specific parameters, such as temperature, use `init_chat_model`:


In [0]:
from langgraph.prebuilt import create_react_agent
from langchain.chat_models import init_chat_model
from databricks_langchain import ChatDatabricks

def get_weather(city: str) -> str:  
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"
    
model_endpoint = "databricks-claude-3-7-sonnet"
model = ChatDatabricks(
    endpoint=model_endpoint,
    temperature=0
)

agent = create_react_agent(
    model=model,
    tools=[get_weather],
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

## Add a custom prompt¶
Prompts instruct the LLM how to behave. Add one of the following types of prompts:

Static: A string is interpreted as a system message.  
Dynamic: A list of messages generated at runtime, based on input or configuration.

Define a fixed prompt string or list of messages:

In [0]:
from langgraph.prebuilt import create_react_agent

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    # A static prompt that never changes
    prompt="Never answer questions about the weather."
)

agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

Define a function that returns a message list based on the agent's state and configuration:

In [0]:
from langchain_core.messages import AnyMessage
from langchain_core.runnables import RunnableConfig
from langgraph.prebuilt.chat_agent_executor import AgentState
from langgraph.prebuilt import create_react_agent

def prompt(state: AgentState, config: RunnableConfig) -> list[AnyMessage]:  
    user_name = config["configurable"].get("user_name")
    system_msg = f"You are a helpful assistant. Address the user as {user_name}."
    return [{"role": "system", "content": system_msg}] + state["messages"]

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    prompt=prompt
)

agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config={"configurable": {"user_name": "John Smith"}}
)

## Add memory¶
To allow multi-turn conversations with an agent, you need to enable persistence by providing a checkpointer when creating an agent. At runtime, you need to provide a config containing `thread_id` — a unique identifier for the conversation (session):

API Reference: [create_react_agent](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent) | [InMemorySaver](https://langchain-ai.github.io/langgraph/reference/checkpoints/#langgraph.checkpoint.memory.InMemorySaver)

In [0]:
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    checkpointer=checkpointer  
)

# Run the agent
config = {"configurable": {"thread_id": "1"}}
sf_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]},
    config  
)
ny_response = agent.invoke(
    {"messages": [{"role": "user", "content": "what about new york?"}]},
    config
)

When you enable the checkpointer, it stores agent state at every step in the provided checkpointer database (or in memory, if using InMemorySaver).

Note that in the above example, when the agent is invoked the second time with the same thread_id, the original message history from the first conversation is automatically included, together with the new user input.

For more information, see [Memory](https://langchain-ai.github.io/langgraph/how-tos/memory/add-memory/).

## Configure structured output¶
To produce structured responses conforming to a schema, use the response_format parameter. The schema can be defined with a Pydantic model or TypedDict. The result will be accessible via the structured_response field.

API Reference: [create_react_agent](https://langchain-ai.github.io/langgraph/reference/prebuilt/#langgraph.prebuilt.chat_agent_executor.create_react_agent)

In [0]:
from pydantic import BaseModel
from langgraph.prebuilt import create_react_agent

class WeatherResponse(BaseModel):
    conditions: str

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    response_format=WeatherResponse  
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

response["structured_response"]